In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle

import statistics
import gc # Helps to clear up some ram

import cv2
from numpy import random
from imgaug import augmenters as img_aug

import math
import os

In [29]:
import pickle

path = "/content/drive/My Drive/AutonomousDriving"
pickle_in = open(path+"/testColour.pickle","rb")
# Remove binary version
testImages = pickle.load(pickle_in)

imageId = []
bigImage = []


for id,image in testImages:
  #checking = image
  if np.count_nonzero(pd.isnull(image)): 
    print("NA: This image was not read well so we will skip for now")
  else:
    imageId.append(id)
    bigImage.append(image)

In [20]:
import pickle

path = "/content/drive/My Drive/AutonomousDriving"
pickle_in = open(path+"/boxTestNumeric.pickle","rb")
# Remove binary version
trainImagesAndLabels = pickle.load(pickle_in)

imageIdTest = []
boxes = []
classes = []
for id,appendage in trainImagesAndLabels:
    imageIdTest.append(id)
    boxes.append(appendage[0])
    classes.append(appendage[1])

# ANGLE FIRST

In [21]:
def shrinkImage(image):
  height, _, _ = image.shape
  image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relevant for lane following
  image = cv2.resize(image, (200,66)) # input image size (200,66) Nvidia model
  return image

In [22]:
X = []
for image in bigImage:
  X.append(shrinkImage(image))

In [23]:
def meanArea(imageClass,box,classID):
  index = np.where(imageClass == classID)[0]
  areaArray = np.zeros(len(index))
  
  
  for _ in range(len(index)):
    idx = index[_]
    area = (box[idx][3]-box[idx][1])*(box[idx][2]-box[idx][0])
    
    areaArray[_] = area

    
  meanArea = np.mean(areaArray)
  return meanArea

In [24]:
def assignSign(imageClass,box):  
  """
  0 = no sign
  1 = right sign
  2 = left sign
  """
  if 6 in imageClass:
    sign = 1
    boxArea = meanArea(imageClass,box,6)
  elif 7 in imageClass:
    sign = 2
    boxArea = meanArea(imageClass,box,7)
  elif 6 and 7 in imageClass:
    print('AHHHHHHHHHHHHHHHH')
  else:
    sign = 0
    boxArea = 0
  return boxArea,sign


## here we can check for redlights as well

In [25]:
def checkRed(imageClass):
  if 4 in imageClass:
    return True
  else:
    pass

In [26]:
boxList = []
signList = []
redLightIndexes = []
for idx in range(len(classes)):
  # boxes,classes = objectDetect(image)
  boxArea, sign = assignSign(classes[idx],boxes[idx])
  boxList.append(boxArea)
  signList.append(sign)
  #
  #check for reds
  if checkRed(classes[idx]):
    redLightIndexes.append(idx)
  


In [37]:
X_new = np.array(X)
imageId = np.array(imageId)
signArray = np.array(signList)
boxArray = np.array(boxList)

In [38]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0) # Gaussian Noise / filtering
    image = image / 255 # normalizing
    # Round everything into the image to 1 decimal place
    image = np.around(image ,2)
    return image

In [39]:
X_final = []
for img_index in range(X_new.shape[0] ):
  X_f = img_preprocess(X_new[img_index,:,:,:])
  X_final.append(X_f)

In [40]:
X_final = np.array(X_final)
X_box_inputs = (np.vstack([signArray,boxArray])).T

In [41]:
model = load_model(path+'/Angle_prediction_model.h5')

In [44]:
steering_angle = model.predict([X_final,X_box_inputs])




Object detect for traffic lights and to create data for speed model



# SPEED PREDICTION




In [45]:
def shrinkImage(image):
  image = image[80:240,:,:]  # remove top half of the image, as it is not relevant for lane following
  image = cv2.resize(image, (0,0), fx=0.6, fy=0.6)# Need to reduce  size to conserve memory
  return image

In [46]:
def img_preprocess(image):
    height, width, _ = image.shape

    # in the case that there exists an object in the image of interest DO NOT MAKE reduce the height of the image!!
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Nvidia model said it is best to use YUV color space
    # image = cv2.GaussianBlur(image, (3,3), 0) # Gaussian Noise / filtering

    image = image / 255 # normalizing
    # Round everything into the image to 1 decimal place
    image = np.around(image ,2)
    return image

In [49]:
bigImage = np.array(bigImage)
X_final = []


for img_index in range(bigImage.shape[0] ):
  X_f = shrinkImage(bigImage[img_index])
  X_f = img_preprocess(X_f)
  X_final.append(X_f)
X_final = np.array(X_final)
if len(X_final.shape) == 3:
  #X_train = tf.reshape(X_train,[ X_train.shape[0],X_train.shape[1],X_train.shape[2] ,1 ]).numpy()
  X_final = np.reshape(X_final, (X_final.shape[0],X_final.shape[1],X_final.shape[2] ,1 ) )
  #X_train = np.repeat(X_train[..., np.newaxis], 3, -1)

In [50]:
model = load_model(path+'/Speed_prediction_model.h5')

In [52]:
steering_angle

array([[0.4033133 ],
       [0.62055427],
       [0.3821722 ],
       ...,
       [0.6534792 ],
       [0.71967554],
       [0.4439296 ]], dtype=float32)

In [53]:
speed = model.predict([X_final,steering_angle])

In [54]:
speed= np.round(speed,0)

In [55]:
for idx in redLightIndexes:
  speed[idx] = 0

In [56]:
submissionFrame = pd.read_csv(path+"/Github/AutonomousCars/data/sampleSubmission.csv")

In [57]:
submissionFrame["angle"] = steering_angle

In [58]:
submissionFrame["image_id"] = imageId

In [59]:
submissionFrame["speed"] = speed

In [60]:
submissionFrame.sort_values(by=['image_id'],inplace=True)

In [61]:
submissionFrame.reset_index(inplace=True,drop=True)

In [ ]:
submissionFrame.head(20)

In [67]:
%cd /content/drive/My Drive/AutonomousDriving/Github/AutonomousCars/data/

/content/drive/My Drive/AutonomousDriving/Github/AutonomousCars/data


In [ ]:
check = pd.read_csv("sampleSubmission.csv")

In [ ]:
np.mean(check['speed']) > np.mean(submissionFrame['speed'])

In [ ]:
check.head(20)

In [ ]:
submissionFrame.head(75)

In [65]:
submissionFrame.to_csv("sampleSubmission6.csv",index=False)

In [ ]:
plt.imshow(boxImages[1])

In [ ]:
submissionFrame.head(75)

,image_id,angle,speed
0,1,0.538020,0.0
1,2,0.771283,1.0
2,3,0.165170,1.0
3,4,0.272043,1.0
4,5,0.200198,1.0
...,...,...,...
70,71,0.461212,1.0
71,72,0.678628,0.0
72,73,0.743421,0.0
73,74,0.697242,0.0
